In [1]:
import os
from constants import *
import pickle as pkl


import pandas as pd
import json
from tqdm import tqdm
import re

from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.document import Document



os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
model_name = paid_source_models[1]

llm = ChatOpenAI(
    model_name= model_name,
    temperature=0.5
)


In [3]:
# model_name = open_source_models[1]

# llm = ChatGroq(
#             model_name = model_name,
#             groq_api_key = groq_api_key,
#             temperature=0.5,
#             max_retries= 10,
#             max_tokens=32768
# )

In [4]:
ranking_prompt = [PromptTemplate.from_template(ranking_prompt_with_tasks[i]) for i in range(len(ranking_prompt_with_tasks))]

In [5]:
df = pd.read_excel(r'./Data/output.xlsx', engine='openpyxl')

collect_docs_summaries = []
for i in range(df.shape[0]):
  collect_docs_summaries.append(f"{df['Use Case ID'][i]}: {df['Use Case Name'][i]} -- {df[f'{paid_source_models[0]}_summaries'][i]}\n")

collect_docs_summaries = "".join(collect_docs_summaries)


In [6]:
all_results = []

for i in tqdm(range(len(ranking_prompt))):
  ranking_chain = ranking_prompt[i] | llm
  ranking_result = ranking_chain.invoke({"texts" : collect_docs_summaries})
  all_results.append(ranking_result.content)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [01:40<00:00, 14.43s/it]


In [13]:
with open("test.txt", "w") as f:
    f.write(all_results[6])

In [23]:
with open(f"{model_name}_outputs1", "wb") as fp:
    pkl.dump(all_results, fp)